This is the project for Machine Learning at UA. 
First, we import the datasets into the notebook after we downloaded the files from the website.

## Data Import

In [ ]:
# import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# read csv files
cleveland_df   = pd.read_csv("processed.cleveland.data", header=None, na_values =["?", -9.0])
switzerland_df = pd.read_csv("processed.switzerland.data", header=None, na_values =["?", -9.0])
va_df          = pd.read_csv("processed.va.data", header=None, na_values =["?", -9.0])
hungarian_df   = pd.read_csv("processed.hungarian.data", header=None, na_values =["?", -9.0])

Now we need to organize the data and add headers to the data frames. 

In [ ]:
#add a column to keep track of the source of the data
cleveland_df["Source"] = "cleveland"
switzerland_df["Source"] = "switzerland"
va_df["Source"] = "va"
hungarian_df["Source"] = "hungarian"

In [ ]:
# add headers to the data frames
headers = {0 : "age",
               1 : "sex",
               2 : "cp",
               3 : "trestbps",
               4 : "chol",
               5 : "fbs",
               6 : "restecg",
               7 : "thalach",
               8 : "exang",
               9 : "oldpeak",
               10 : "slope",
               11 : "ca",
               12 : "thal",
               13 : "diagnosis"}

cleveland_df = cleveland_df.rename(columns=headers)
switzerland_df = switzerland_df.rename(columns=headers)
va_df = va_df.rename(columns=headers)
hungarian_df = hungarian_df.rename(columns=headers)

Now we want to combine the different datasets from Cleveland, Switzerland, Va and Hungary into one big dataset. 

In [ ]:
heart_disease_df = pd.concat([cleveland_df, switzerland_df, va_df, hungarian_df])

Next, we want to look at some details of the dataset. 

In [ ]:
print(heart_disease_df.head())
print(heart_disease_df.info())

Next, let's use the command from the lecture to describe the data. 

In [ ]:
heart_disease_df.describe()

Let's collect some more information concerning the dataset. 


In [ ]:
print(heart_disease_df.shape)
print(heart_disease_df.size)

Some more analytics. There are 920 observations with 15 14 features each, resulting in one diagnosis.

In [ ]:
heart_disease_df.corr(method = 'pearson')

Analyse the Pearson correlation coefficient, to find connections within the dataset. 

In [ ]:
plt.matshow(heart_disease_df.corr())
plt.show()

In [ ]:
rs = np.random.RandomState(0)
df = pd.DataFrame(rs.rand(10, 10))
corr = heart_disease_df.corr()
corr.style.background_gradient(cmap='coolwarm')

## Data Visualization

In [ ]:
sns.set()
heart_disease_df.hist(figsize=(10,10))
plt.show()
plt.savefig('heart_disease_hist.png', dpi=300, bbox_inches='tight')

In [ ]:
heart_disease_df.head()
sns.set_style('whitegrid')
sns.histplot(heart_disease_df['diagnosis'], kde = False, color ='red', bins = 30)

In [ ]:
# Define the age ranges for grouping
age_bins = [20, 30, 40, 50, 60, 70, 80]

# Create a new column 'age_group' based on age bins
heart_disease_df['age_group'] = pd.cut(heart_disease_df['age'], bins=age_bins)

# Create the boxplot with age group on x-axis and diagnosis on y-axis
sns.boxplot(x='age_group', y='diagnosis', data=heart_disease_df)
heart_disease_df= heart_disease_df.drop('age_group', axis=1)

In [ ]:
'''
# Group the data by age and calculate the mean, median of a column for each group
age_stats_df = heart_disease_df.groupby('age')['diagnosis'].agg(['mean', 'median']).reset_index()

# Plot the graph using seaborn lineplot
sns.lineplot(x='age', y='mean', data=age_stats_df, label='Average')
sns.lineplot(x='age', y='median', data=age_stats_df, label='Median')
sns.lineplot(x='age', y='diagnosis', data=heart_disease_df, estimator=None, alpha=0.2, color='gray')
'''

In [ ]:
sns.boxplot(x='ca',y='diagnosis',data=heart_disease_df)

## Pre-Processing

In [ ]:
import os
import pandas as pd
import sklearn
import sklearn.model_selection
import sklearn.preprocessing
import sklearn.pipeline
import numpy as np
from sklearn.impute import SimpleImputer 

In [ ]:
CONTINUOUS_FACTORS = ["age", "chol", "oldpeak", "thalach", "trestbps"]
DISCRETE_FACTORS = ["ca", "cp", "exang", "fbs", "restecg", "sex", "slope", "thal"]
TARGET = ["diagnosis"]

 
train, test = sklearn.model_selection.train_test_split(heart_disease_df, test_size=.2, random_state=10)

# split the data into the X and Y variables
trainY = train["diagnosis"].copy()
trainX = train.drop("diagnosis", axis=1)
testY = test["diagnosis"].copy()
testX = test.drop("diagnosis", axis=1)

# Handle the discrete and continuous variables seperatly
trainX_continuous = trainX[CONTINUOUS_FACTORS]
trainX_discrete = trainX[DISCRETE_FACTORS].fillna(value=5) #For all the discrete values, replace missing values with 5
testX_continuous = testX[CONTINUOUS_FACTORS]
testX_discrete = testX[DISCRETE_FACTORS].fillna(value=5) #For all the discrete values, replace missing values with 5

In [ ]:
 # For continuous variables, replace missing values with the median and then normalize by subtracting the mean and dividing by the standard deviation
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

continuous_Pipeline = sklearn.pipeline.Pipeline( [("imputer", sklearn.impute.SimpleImputer(strategy="median")),
                                             ("scaler", sklearn.preprocessing.StandardScaler())
                                            ]
                                           )
trainX_continuous_scaled = continuous_Pipeline.fit_transform(trainX_continuous)

In [ ]:
# for discrete variables, one-hot encode the data
discrete_Pipeline = sklearn.pipeline.Pipeline( [("one_hot", sklearn.preprocessing.OneHotEncoder(handle_unknown ='ignore'))
                                            ]
                                           )
trainX_discrete_one_hot = discrete_Pipeline.fit_transform(trainX_discrete)

print(np.size(trainX_discrete, 1))
print(np.size(trainX_discrete_one_hot, 1))


The one hot encoding increases the number of features from 8 to 29, since artificially zeros are introduced. 

In [ ]:
trainX_fully_preprocessed = np.concatenate((trainX_continuous_scaled, trainX_discrete_one_hot.toarray()), axis = 1)

In [ ]:
# preprocess the test data, note the use of transform instead of fit_transform
testX_continuous_scaled = continuous_Pipeline.transform(testX_continuous)
testX_discrete_one_hot = discrete_Pipeline.transform(testX_discrete)
testX_fully_preprocessed = np.concatenate((testX_continuous_scaled, testX_discrete_one_hot.toarray()), axis = 1)

In [ ]:
# Convert the Y varialbe to a binary varible
trainY_binary = (trainY>0) # True if they have heart disease, False otherwide
testY_binary = (testY>0) # True if they have heart disease, False otherwide

Preprocessing done. There are two datasets: trainY_binary and trainY. 

## Train models

### Linear Regression

Let's write 2 helper functions of calculating and printing evaluation metrics.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculate and print the evaluation metrics
def print_evaluation_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f'Mean Squared Error: {mse:.4f}')
    print(f'R-squared: {r2:.4f}')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Calculate and print the classification metrics
def print_classification_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'ROC AUC Score: {roc_auc:.4f}')

In [ ]:
print(trainX_fully_preprocessed.shape)
print(testX_fully_preprocessed.shape)
print(trainY_binary.shape)
print(testY_binary.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

# Create a linear regression model
linear_reg = LinearRegression()

# Fit the model to the training data
linear_reg.fit(trainX_fully_preprocessed, trainY_binary)

# Predict the target values for the train and test data
trainY_pred = linear_reg.predict(trainX_fully_preprocessed)
testY_pred = linear_reg.predict(testX_fully_preprocessed)

# Since linear regression returns continuous values, we need to convert the predictions into binary classification
lin_trainY_pred_binary = (trainY_pred >= 0.5).astype(int)
lin_testY_pred_binary = (testY_pred >= 0.5).astype(int)

# Calculate and print the evaluation and classification metrics
print("TRAINING DATA:")
print_evaluation_metrics(trainY_binary, trainY_pred)
print_classification_metrics(trainY_binary, lin_trainY_pred_binary)
print("\nTEST DATA:")
print_evaluation_metrics(testY_binary, testY_pred)
print_classification_metrics(testY_binary, lin_testY_pred_binary)

Next, we try to optimise the type of the regularization and the regularization strength .

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

# Assuming trainX_fully_preprocessed, trainY_binary are loaded as NumPy arrays or pandas DataFrames

# Define the hyperparameters to optimize
params = {
    'alpha': np.logspace(-4, 4, 9)  # Regularization strength
}

# Create Ridge and Lasso models
ridge = Ridge()
lasso = Lasso()

# Create GridSearchCV objects for Ridge and Lasso
ridge_grid = GridSearchCV(ridge, params, scoring='neg_mean_squared_error', cv=5)
lasso_grid = GridSearchCV(lasso, params, scoring='neg_mean_squared_error', cv=5)

# Fit the GridSearchCV objects to the training data
ridge_grid.fit(trainX_fully_preprocessed, trainY_binary)
lasso_grid.fit(trainX_fully_preprocessed, trainY_binary)

# Get the best hyperparameters and models
best_ridge_params = ridge_grid.best_params_
best_lasso_params = lasso_grid.best_params_
best_ridge_model = ridge_grid.best_estimator_
best_lasso_model = lasso_grid.best_estimator_

print(f'Best Ridge parameters: {best_ridge_params}')
print(f'Best Lasso parameters: {best_lasso_params}')

In [ ]:
# Make predictions using the best Ridge and Lasso models
ridge_testY_pred = best_ridge_model.predict(testX_fully_preprocessed)
lasso_testY_pred = best_lasso_model.predict(testX_fully_preprocessed)

# Convert the continuous predictions into binary classification
ridge_testY_pred_binary = (ridge_testY_pred >= 0.5).astype(int)
lasso_testY_pred_binary = (lasso_testY_pred >= 0.5).astype(int)

# Calculate and print the evaluation metrics for Ridge model
print("Ridge Model:")
print_evaluation_metrics(testY_binary, ridge_testY_pred)
print_classification_metrics(testY_binary, ridge_testY_pred_binary)

# Calculate and print the evaluation metrics for Lasso model
print("\nLasso Model:")
print_evaluation_metrics(testY_binary, lasso_testY_pred)
print_classification_metrics(testY_binary, lasso_testY_pred_binary)

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create a logistic regression model
logistic_reg = LogisticRegression(solver='saga', max_iter=3000, verbose=1)

# Fit the model to the training data
logistic_reg.fit(trainX_fully_preprocessed, trainY_binary)

# Predict the target values for the training and test data
trainY_pred = logistic_reg.predict(trainX_fully_preprocessed)
testY_pred = logistic_reg.predict(testX_fully_preprocessed)

# Calculate and print the evaluation metrics
print("TRAIN DATA:")
print_classification_metrics(trainY_binary, trainY_pred)
print("\nTEST DATA:")
print_classification_metrics(testY_binary, testY_pred)

In [ ]:
# Define the hyperparameters to optimize
params = {
    'C': np.logspace(-4, 4, 9),  # Inverse of regularization strength
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Create a logistic regression model
logistic_reg = LogisticRegression(max_iter=3000)

# Create a GridSearchCV object for logistic regression
logistic_grid = GridSearchCV(logistic_reg, params, scoring='accuracy', cv=5)

# Fit the GridSearchCV object to the training data
logistic_grid.fit(trainX_fully_preprocessed, trainY_binary)

# Get the best hyperparameters and model
best_logistic_params = logistic_grid.best_params_
best_logistic_model = logistic_grid.best_estimator_

print(f'Best logistic regression parameters: {best_logistic_params}')

In [ ]:
# Predict the target values for the training and test data
logistic_trainY_pred = best_logistic_model.predict(trainX_fully_preprocessed)
logistic_testY_pred = best_logistic_model.predict(testX_fully_preprocessed)

# Calculate and print the evaluation metrics
print("TRAIN DATA:")
print_classification_metrics(trainY_binary, logistic_trainY_pred)
print("\nTEST DATA:")
print_classification_metrics(testY_binary, logistic_testY_pred)

### Support Vector Machines


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Create a Support vector model
svm = SVC()

# Fit the model to the training data
svm.fit(trainX_fully_preprocessed, trainY_binary)

# Predict the target values for the test data
testY_pred = svm.predict(testX_fully_preprocessed)

# Calculate and print the evaluation metrics
print_classification_metrics(testY_binary, testY_pred)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to optimize
params = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],  # Kernel function
    'gamma': ['scale', 'auto']  # Kernel coefficient for rbf, poly and sigmoid
}

# Create a GridSearchCV object for SVM
svm_grid = GridSearchCV(svm, params, scoring='accuracy', cv=5)

# Fit the GridSearchCV object to the training data
svm_grid.fit(trainX_fully_preprocessed, trainY_binary)

# Get the best hyperparameters and model
best_svm_params = svm_grid.best_params_
best_svm_model = svm_grid.best_estimator_


print(f'Best SVM parameters: {best_svm_params}')

In [ ]:
svm_testY_pred = best_svm_model.predict(testX_fully_preprocessed)

print_classification_metrics(testY_binary, svm_testY_pred)

### Neural Network classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

# Create a neural network classifier model
mlp = MLPClassifier(max_iter = 2000, solver = 'adam')

# Fit the model to the training data
mlp.fit(trainX_fully_preprocessed, trainY_binary)

# Predict the target values for the test data
trainY_pred_nn = mlp.predict(trainX_fully_preprocessed)
testY_pred_nn = mlp.predict(testX_fully_preprocessed)

# Calculate and print the evaluation metrics
print("TRAIN DATA:")
print_classification_metrics(trainY_binary, trainY_pred_nn)
print("\nTEST DATA:")
print_classification_metrics(testY_binary, testY_pred_nn)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to optimize
params = {
    'hidden_layer_sizes': [(34,), (17,), (34, 17), (17, 8), (17, 8, 4), (34, 17, 8)],
    'alpha': [0.001, 0.01, 0.1, 1],
    'activation': ['relu'],
    'learning_rate_init': [0.1],
    'batch_size': [16],  # Different batch sizes
}

# Create a neural network classifier model
mlp = MLPClassifier(max_iter=500, solver='adam', early_stopping=False)

# Create a GridSearchCV object for the neural network classifier
mlp_grid = GridSearchCV(mlp, params, scoring='accuracy', cv=5)

# Fit the GridSearchCV object to the training data
mlp_grid.fit(trainX_fully_preprocessed, trainY_binary)

# Get the best hyperparameters and model
best_mlp_params = mlp_grid.best_params_
best_mlp_model = mlp_grid.best_estimator_

print(f'Best NN parameters: {best_mlp_params}')

In [ ]:
neural_network_trainY_pred = best_mlp_model.predict(trainX_fully_preprocessed)
neural_network_testY_pred = best_mlp_model.predict(testX_fully_preprocessed)

print("TRAIN DATA:")
print_classification_metrics(trainY_binary, neural_network_trainY_pred)
print("\nTEST DATA:")
print_classification_metrics(testY_binary, neural_network_testY_pred)

In [ ]:
# Plot the cost function trajectory over iterations
plt.plot(best_mlp_model.loss_curve_)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.title('Cost Function Trajectory Over Iterations')
plt.show()

Compare results of all classifications only for optimized hyperparameters.

In [ ]:
# lin regression
print('lin reg lasso')
# print_classification_metrics(testY_binary, lasso_testY_pred_binary)
print('')
print('lin reg ridge')
# print_classification_metrics(testY_binary, ridge_testY_pred_binary)
print('')
# log regression
print('log reg')
# print_classification_metrics(testY_binary, logistic_testY_pred)
print('')
# svm
print('svm')
# print_classification_metrics(testY_binary, svm_testY_pred)
print('')
# nn
print('neural network')
# print_classification_metrics(testY_binary, neural_network_testY_pred)

In [ ]:
def evaluate_classification_model(model, testX, testY, threshold=0.5):
    '''
    Evaluates a binary classification model on the test set
    Returns a dictionary with the evaluation metrics
    '''
    predY = model.predict(testX) > threshold
    
    acc = accuracy_score(testY, predY)
    prec = precision_score(testY, predY)
    rec = recall_score(testY, predY)
    f1 = f1_score(testY, predY)
    roc_auc = roc_auc_score(testY, predY)
    
    return {'Accuracy': acc,
            'Precision': prec,
            'Recall': rec,
            'F1 Score': f1,
            'ROC AUC Score': roc_auc}

In [ ]:
from tabulate import tabulate

# Evaluation metrics dictionary for each model
lasso_metrics = evaluate_classification_model(best_lasso_model, testX_fully_preprocessed, testY_binary)
ridge_metrics = evaluate_classification_model(best_ridge_model, testX_fully_preprocessed, testY_binary)
logistic_metrics = evaluate_classification_model(best_logistic_model, testX_fully_preprocessed, testY_binary)
svm_metrics = evaluate_classification_model(best_svm_model, testX_fully_preprocessed, testY_binary)
nn_metrics = evaluate_classification_model(best_mlp_model, testX_fully_preprocessed, testY_binary)

# Create a pandas DataFrame for the evaluation metrics
results = pd.DataFrame({'Model': ['Lasso Regression', 'Ridge Regression', 'Logistic Regression', 'SVM', 'Neural Network'],
                   'Accuracy': [lasso_metrics['Accuracy'], ridge_metrics['Accuracy'], logistic_metrics['Accuracy'], svm_metrics['Accuracy'], nn_metrics['Accuracy']],
                   'Precision': [lasso_metrics['Precision'], ridge_metrics['Precision'], logistic_metrics['Precision'], svm_metrics['Precision'], nn_metrics['Precision']],
                   'Recall': [lasso_metrics['Recall'], ridge_metrics['Recall'], logistic_metrics['Recall'], svm_metrics['Recall'], nn_metrics['Recall']],
                   'F1 Score': [lasso_metrics['F1 Score'], ridge_metrics['F1 Score'], logistic_metrics['F1 Score'], svm_metrics['F1 Score'], nn_metrics['F1 Score']],
                   'ROC AUC Score': [lasso_metrics['ROC AUC Score'], ridge_metrics['ROC AUC Score'], logistic_metrics['ROC AUC Score'], svm_metrics['ROC AUC Score'], nn_metrics['ROC AUC Score']]})


df = pd.DataFrame(results)
print(df)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

# Define the models you want to evaluate
models = [('lin reg lasso', best_lasso_model),
          ('lin reg ridge', best_ridge_model),
          ('log reg', best_logistic_model),
          ('svm', best_svm_model),
          ('neural network', best_mlp_model)]

In [ ]:
cm_lasso = confusion_matrix(testY_binary,lasso_testY_pred_binary)
cm_ridge = confusion_matrix(testY_binary,ridge_testY_pred_binary)
cm_log = confusion_matrix(testY_binary,logistic_testY_pred)
cm_svm = confusion_matrix(testY_binary,svm_testY_pred)
cm_nn = confusion_matrix(testY_binary,neural_network_testY_pred)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm_lasso.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm_lasso.flatten()/np.sum(cm_lasso)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm_lasso, annot=labels, fmt="", cmap='Blues')

In [ ]:
fig = plt.figure(figsize= (6,4))
fig.suptitle('All confusion matrices')
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm_ridge.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm_ridge.flatten()/np.sum(cm_ridge)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm_ridge, annot=labels, fmt="", cmap='Blues')

In [ ]:
fig.suptitle('All confusion matrices')
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm_log.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm_log.flatten()/np.sum(cm_log)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm_log, annot=labels, fmt="", cmap='Blues')



In [ ]:
fig.suptitle('All confusion matrices')
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm_svm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm_svm.flatten()/np.sum(cm_svm)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm_svm, annot=labels, fmt="", cmap='Blues')


In [ ]:
fig.suptitle('All confusion matrices')
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm_nn.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm_nn.flatten()/np.sum(cm_nn)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm_nn, annot=labels, fmt="", cmap='Blues')

# from here old code starts which does not follow our current procedure

### SVM (old)

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

# train a SVM model using library
classifier = SVC(kernel='rbf',random_state=1,C=1,gamma='auto')
classifier.fit(trainX_fully_preprocessed,trainY)

# perform prediction on x_test data
y_pred = classifier.predict(testX_fully_preprocessed)

cm = confusion_matrix(testY,y_pred)
print(cm)
accuracy = float(cm.diagonal().sum())/len(testY)
print('model accuracy is:',accuracy*100,'%')



In [ ]:
# now use the binary data to see whether the prediction is more easibly possible
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

# train a SVM model using library
classifier = SVC(kernel='rbf',random_state=1,C=1,gamma='auto')
classifier.fit(trainX_fully_preprocessed,trainY_binary)

# perform prediction on x_test data
y_pred_binary = classifier.predict(testX_fully_preprocessed)

cm = confusion_matrix(testY_binary,y_pred_binary)
print(cm)
accuracy = float(cm.diagonal().sum())/len(testY)
print('model accuracy is:',accuracy*100,'%')

# Calculate and print the evaluation metrics
print_classification_metrics(testY_binary, y_pred_binary)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm, annot=labels, fmt="", cmap='Blues')

## Neural network from class

Define activation function

In [ ]:
def sigmoidGradient(z):
    """
    computes the gradient of the sigmoid function
    """
    sigmoid = 1/(1 + np.exp(-z))
    
    return sigmoid *(1-sigmoid) 


def sigmoid(z):
    """
    return the sigmoid of z
    """
    
    return 1/ (1 + np.exp(-z))

Define cost function

Inputs for cost function:
nn_params
input_layer_size
hidden_layer_size
num_labels
X
y
Lambda

In [ ]:
def nnCostFunction(nn_params,input_layer_size, hidden_layer_size, num_labels, X, y,Lambda):
    """
    nn_params contains the parameters unrolled into a vector
    
    compute the cost and gradient of the neural network
    """
    # Reshape nn_params back into the parameters Theta1 and Theta2
    Theta1 = nn_params[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
    Theta2 = nn_params[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)
    
    m = X.shape[0]
    J=0
    X = np.hstack((np.ones((m,1)),X))
    y10 = np.zeros((m,num_labels))
    
    a1 = sigmoid(X @ Theta1.T)
    a1 = np.hstack((np.ones((m,1)), a1)) # hidden layer
    a2 = sigmoid(a1 @ Theta2.T) # output layer
    
    for i in range(1,num_labels+1):
        # y10[:,i-1][:,np.newaxis] = np.where(y==i,1,0)
        y10[:, i-1] = np.where(y==i, 1, 0)
    for j in range(num_labels):
        J = J + sum(-y10[:,j] * np.log(a2[:,j]) - (1-y10[:,j])*np.log(1-a2[:,j]))
    
    cost = 1/m* J
    reg_J = cost + Lambda/(2*m) * (np.sum(Theta1[:,1:]**2) + np.sum(Theta2[:,1:]**2))
    
    # Implement the backpropagation algorithm to compute the gradients
    
    grad1 = np.zeros((Theta1.shape))
    grad2 = np.zeros((Theta2.shape))
    
    for i in range(m):
        xi= X[i,:] # 1 X 401
        a1i = a1[i,:] # 1 X 26
        a2i =a2[i,:] # 1 X 10
        d2 = a2i - y10[i,:]
        d1 = Theta2.T @ d2.T * sigmoidGradient(np.hstack((1,xi @ Theta1.T)))
        grad1= grad1 + d1[1:][:,np.newaxis] @ xi[:,np.newaxis].T
        grad2 = grad2 + d2.T[:,np.newaxis] @ a1i[:,np.newaxis].T
        
    grad1 = 1/m * grad1
    grad2 = 1/m*grad2
    
    grad1_reg = grad1 + (Lambda/m) * np.hstack((np.zeros((Theta1.shape[0],1)),Theta1[:,1:]))
    grad2_reg = grad2 + (Lambda/m) * np.hstack((np.zeros((Theta2.shape[0],1)),Theta2[:,1:]))
    
    return cost, grad1, grad2, reg_J, grad1_reg, grad2_reg


Random initialization

In [ ]:
def randInitializeWeights(L_in, L_out):
    """
    randomly initializes the weights of a layer with L_in incoming connections and L_out outgoing connections.
    """
    
    epi = (6**1/2) / (L_in + L_out)**1/2
    
    W = np.random.rand(L_out,L_in +1) *(2*epi) -epi
    
    return W

Learning parameters using gradient descent

In [ ]:
def gradientDescentnn(X,y,initial_nn_params,alpha,num_iters,Lambda,input_layer_size, hidden_layer_size, num_labels):
    """
    Take in numpy array X, y and theta and update theta by taking num_iters gradient steps
    with learning rate of alpha
    
    return theta and the list of the cost of theta during each iteration
    """
    Theta1 = initial_nn_params[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
    Theta2 = initial_nn_params[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)
    
    m=len(y)
    J_history =[]
    
    for i in range(num_iters):
        nn_params = np.append(Theta1.flatten(),Theta2.flatten())
        cost, grad1, grad2 = nnCostFunction(nn_params,input_layer_size, hidden_layer_size, num_labels,X, y,Lambda)[3:]
        Theta1 = Theta1 - (alpha * grad1)
        Theta2 = Theta2 - (alpha * grad2)
        J_history.append(cost)
    
    nn_paramsFinal = np.append(Theta1.flatten(),Theta2.flatten())
    return nn_paramsFinal , J_history

Now we need to define the parameters for our neural network. 

In [ ]:
input_layer_size  = 34
hidden_layer_size = 50
num_labels = 2
num_labels_binary = 2
alpha=0.001 #learning rate
num_iters=1000
Lambda=0.1

# X = trainX_fully_preprocessed
# y = trainY
# y_binary = trainY_binary

In [ ]:
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

initial_nn_params = np.append(initial_Theta1.flatten(),initial_Theta2.flatten())

nnTheta, nnJ_history = gradientDescentnn(trainX_fully_preprocessed,trainY_binary,initial_nn_params,alpha,num_iters,Lambda,input_layer_size, hidden_layer_size, num_labels)

Theta1 = nnTheta[:((input_layer_size+1) * hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
Theta2 = nnTheta[((input_layer_size +1)* hidden_layer_size ):].reshape(num_labels,hidden_layer_size+1)

In [ ]:
# Plot the cost function evolution during training.
#In order to say learning has finished, the cost function has to converge to a flat rate
plt.plot(nnJ_history)  #
plt.xlabel("Iteration")
plt.ylabel("$J(\Theta)$")
plt.title("Cost function using Gradient Descent")

Prediction

In [ ]:
def predict(Theta1, Theta2, X):
    """
    Predict the label of an input given a trained neural network
    """
    m= X.shape[0]
    #Add a column of ones
  
    X = np.append(np.ones((m,1)),X, axis=1)
    
    #Compute the output of the hidden layer (with sigmoid activation functions)
    z1=np.dot(X, Theta1.T)
    a1=sigmoid(z1)
    
    #Add a column of ones
    a1 = np.append(np.ones((m,1)),a1, axis=1)
    
    #Compute the output of the output layer (with sigmoid activation functions)
    z2=np.dot(a1, Theta2.T)
    a2=sigmoid(z2)
    
    return np.argmax(a2,axis=1)+1

In [ ]:
pred3 = predict(Theta1, Theta2, testX_fully_preprocessed)

m = len(testY)
pred3 = pred3.reshape(m,1)
trainY_array = testY_binary.values.reshape(m,1)

print("Training Set Accuracy:",sum(pred3==trainY_array)/m*100,"%")

## Neural Network

In [ ]:
from sklearn.neural_network import MLPClassifier

# important 
# trainX_fully_preprocessed
# testX_fully_preprocessed

# trainY
# testY

# trainY_binary
# testY_binary

In [ ]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(34, 10), max_iter = 2000, random_state=1)

clf.fit(trainX_fully_preprocessed,trainY)


In [ ]:
nn_pred = clf.predict(trainX_fully_preprocessed)

m = len(trainY)
nn_pred = nn_pred.reshape(m,1)
trainY_array = trainY.values.reshape(m,1)

print("Training Set Accuracy:",sum(nn_pred==trainY_array)/m*100,"%")

In [ ]:
nn_pred_test = clf.predict(testX_fully_preprocessed)

m_test = len(testY)
nn_pred_test = nn_pred_test.reshape(m_test,1)
trainY_array = testY.values.reshape(m_test,1)

print("Training Set Accuracy:",sum(nn_pred_test==trainY_array)/m_test*100,"%")

In [ ]:
# plt.subplot(3, 1, 1) # row 1, col 2 index 1
# plt.plot(nn_pred_test)
# plt.subplot(3, 1, 2) # row 1, col 2 index 1
# plt.plot(testY_array)
# plt.subplot(3, 1, 3) # row 1, col 2 index 1
# plt.plot(nn_pred_test)
# plt.plot(testY_array)
cm = confusion_matrix(testY,nn_pred_test)
print(cm)
accuracy = float(cm.diagonal().sum())/len(testY)
print('model accuracy is:',accuracy*100,'%')

### Binary classification (neural networks - sklearn)

In [ ]:
clf_binary = MLPClassifier(solver='lbfgs', alpha=1e-6,hidden_layer_sizes=(34, 10), max_iter = 5000, random_state=1)

clf_binary.fit(trainX_fully_preprocessed,trainY_binary)

In [ ]:
# nn_pred_binary = clf_binary.predict(trainX_fully_preprocessed)
# m_binary = len(trainY_binary)
# nn_pred_binary = nn_pred_binary.reshape(m_binary,1)
# trainY_array_binary = trainY_binary.values.reshape(m_binary,1)
# print("Training Set Accuracy:",sum(nn_pred_binary==trainY_array_binary)/m_binary*100,"%")

In [ ]:
nn_pred_binary_test = clf_binary.predict(testX_fully_preprocessed)
m_binary_test = len(testY_binary)
nn_pred_binary_test = nn_pred_binary_test.reshape(m_binary_test,1)
testY_array= testY_binary.values.reshape(m_binary_test,1)

print("Training Set Accuracy:",sum(nn_pred_binary_test==testY_array)/m_binary_test*100,"%")

In [ ]:
cm = confusion_matrix(testY_binary,nn_pred_binary_test)
print(cm)
accuracy = float(cm.diagonal().sum())/len(testY)
print('model accuracy is:',accuracy*100,'%')

print_classification_metrics(testY_binary, nn_pred_binary_test)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm, annot=labels, fmt="", cmap='Blues')

Neural network - clean

In [ ]:
from sklearn.neural_network import MLPClassifier

# Create a neural  model
linear_reg = LinearRegression()

# Fit the model to the training data
linear_reg.fit(trainX_fully_preprocessed, trainY_binary)

# Predict the target values for the test data
testY_pred = linear_reg.predict(testX_fully_preprocessed)

# Since linear regression returns continuous values, we need to convert the predictions into binary classification
testY_pred_binary = (testY_pred >= 0.5).astype(int)

# Calculate and print the evaluation and classification metrics
print_evaluation_metrics(testY_binary, testY_pred)
print_classification_metrics(testY_binary, testY_pred_binary)

## use this for visualizing the confusion matrix

In [ ]:
group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ["{0:0.0f}".format(value) for value in
                cm.flatten()]
group_percentages = ["{0:.2%}".format(value) for value in
                     cm.flatten()/np.sum(cm)]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm, annot=labels, fmt="", cmap='Blues')

In [ ]:
labels = ['True Neg','False Pos','False Neg','True Pos']
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cm, annot=labels, fmt="", cmap='Blues')